# Notebook machine learning
Entrenamiento de modelos de Regresión Lineal y Random Forest para predecir la velocidad media de la vuelta rápida a partir del tiempo de vuelta

Importamos bibliotecas necesarias


In [ ]:
import requests
import csv
import pandas as pd
import json
import ast
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Scikit-Learn 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics


Función para convertir los tiempos de vuelta con formato mm:ss.ffff a segundos

In [ ]:

def duration_to_seconds(duration):
    duration_str = str(duration)
    if ':' in duration_str: 
        minutes, seconds_miliseconds = duration_str.split(':')
        seconds, miliseconds = seconds_miliseconds.split('.')
        return int(minutes) * 60 + int(seconds) + int(miliseconds) / 1000
    else:  
        return float(duration_str)

### Carga de datos y limpieza

In [ ]:
raw_races_df = pd.read_csv('../data/results_2000-2024.csv')
raw_races_df.head()

Una vez cargado el CSV en un dataframe, se procede a convertir las columnas Results y Circuit a estructuras de Python. <br />
Primero, se expande la columna Results, creando una lista llamada rows que contiene los datos generales de las carreras por cada participante, sin la columna Results, y otra lista results_expanded con la información de cada participante, es decir, el contenido de la columna Results. El objetivo es tener una fila por cada participante. <br />
Una vez terminado los bucles, se convierten las listas a dataframes, aplanando las estructuras anidadas en results_expanded mediante json_normalize. <br />
Se genera un nuevo dataframe a partir de la concatenación de los dos dataframes anteriores, obteniendo una fila por cada participante con su información así como información de la carrera. <br />
Por último, se normaliza la columna Circuit que también contiene una estructura anidada.


In [ ]:
raw_races_df['Results'] = raw_races_df['Results'].apply(ast.literal_eval)
raw_races_df['Circuit'] = raw_races_df['Circuit'].apply(ast.literal_eval)


rows = []
results_expanded = []
for index, row in raw_races_df.iterrows():
    for result in row['Results']:
        rows.append(row.drop('Results')) 
        results_expanded.append(result)  

expanded_rows_df = pd.DataFrame(rows)
results_normalized_df = pd.json_normalize(results_expanded) # Se normaliza columna Results
race_results_df = pd.concat([expanded_rows_df.reset_index(drop=True), results_normalized_df.reset_index(drop=True)], axis=1) # Se añade la columna Results normalizadas al dataframe donde se contiene la información de carrera

circuits_normalized = pd.json_normalize(race_results_df['Circuit']) # Se normaliza columna Circuit
race_results_df = pd.concat([race_results_df.drop(columns=['Circuit']), circuits_normalized],axis=1) # Se añaden la columna Circuit normalizada al dataframe

race_results_df.head()

In [ ]:
race_results_df.columns

Borramos columnas innecesarias

In [ ]:
modified_race_results_df = race_results_df.drop(columns=['url', 'time', 'Location.country', 'Location.lat', 'Location.long', 'url', 'Driver.permanentNumber', 'Constructor.nationality', 'Constructor.constructorId', 'Constructor.url', 'circuitId', 'Location.locality', 'positionText', 'points', 'Driver.nationality', 'Driver.dateOfBirth', 'Time.time', 'Time.millis', 'Driver.url', 'Driver.driverId', 'Driver.code'])
modified_race_results_df[modified_race_results_df['season'] == 2024].tail()

Renombramos las variables que vamos a usar en el modelo y creamos una nueva columna que contenga el tiempo de vuelta en segundos mediante la función duration_to_seconds que se creó al principio del notebook.

In [ ]:
renamed_race_results_df = modified_race_results_df.rename(columns= {'FastestLap.Time.time': 'FastestLapTime', 'FastestLap.AverageSpeed.speed': 'FastestLapAvgSpeed'})
renamed_race_results_df['FastestLapTimeSeconds'] = renamed_race_results_df['FastestLapTime'].apply(lambda duration: duration_to_seconds(duration))
renamed_race_results_df.head()

Comprobamos cuantos valores NaN contiene el dataframe

In [ ]:
print("Valores NA:")
print(renamed_race_results_df.isna().sum())

Convertimos columnas a variables numéricas

In [ ]:
renamed_race_results_df['FastestLapAvgSpeed'] = pd.to_numeric(renamed_race_results_df['FastestLapAvgSpeed'], errors='coerce')
renamed_race_results_df['grid'] = renamed_race_results_df['grid'].astype(str).astype(int)
renamed_race_results_df['position'] = renamed_race_results_df['position'].astype(str).astype(int)
renamed_race_results_df.dtypes

Borramos registros con valores NaN

In [ ]:
renamed_race_results_df = renamed_race_results_df.dropna()

Pintamos matriz de correlación para conocer las relaciones entre las distintas columnas numéricas que disponemos en el dataframe

In [ ]:
speed_corr = renamed_race_results_df.corr(numeric_only=True)

fig, ax = plt.subplots(figsize=(12, 10))

sns.heatmap(speed_corr, annot=True, fmt=".2f")

Borrado de columnas innecesarias

In [ ]:
tmp_renamed_race_results_df = renamed_race_results_df.drop(columns=['raceName', 'round', 'date', 'number', 'laps', 'status', 'Driver.givenName', 'Driver.familyName', 'Constructor.name', 'FastestLap.rank', 'FastestLap.lap', 'FastestLapTime', 'FastestLap.AverageSpeed.units'])
tmp_renamed_race_results_df.tail()

Comprobamos que no hayan valores nulos

In [ ]:
print(tmp_renamed_race_results_df.isna().sum())


Generamos una copia del dataframe modificado, y este será el dataframe a usar en los modelos

In [ ]:
final_race_results_df = tmp_renamed_race_results_df.copy()

In [ ]:
print(final_race_results_df['circuitName'].unique())

### Modelos

#### Linear Regression - Fastest Lap vs Fastest Lap Avg Speed (Todos los circuitos) 

Implementación de un modelo de Regresión Lineal Multiple, donde tenemos varias variables independientes. El objetivo es predecir la velocidad promedio de la vuelta rápida a partir del tiempo de vuelta rápida. <br />
En primer lugar, convertimos la variable categórica circuito, en formato binario donde habrá una columna por cada circuito, que será relevante para entrenar el modelo y calcular las predicciones. <br />
A pesar de tener datos desde el año 2000, se ha decidido trabajar con los datos a partir del año 2014 donde se produjo el salto a la era híbrida en la F1. Para entrenamiento se seleccionan los datos que van desde el 2014 hasta el 2021, y para probar el modelo los años 2022 y 2023. <br />
Se define como variable dependiente el FastestLapAvgSpeed en y, y como variable dependientes en x contamos con position, grid, FastestLapTimeSeconds y las columnas binarias que representan cada circuito.<br />
Se entrena el modelo y se obtienen las predicciones. A partir de los resultados de test y las predicciones, se muestra el intercept, coeficiente y RMSE (Root Mean Squared Error).
Por último, se pinta un gráfico de dispersión que permite comparar los datos reales (y_test) con las predicciones del modelo (y_pred), mostrando cómo se relacionan los tiempos de vuelta más rápidos con las velocidades promedio. Además, existe otro gráfico donde se pintan los valores reales en x frente a los predichos en y.

In [ ]:
# One-hot encoder para convertir variables categóricas en formato numérico para poder ser utilizadas por el modelo
enc = OneHotEncoder(sparse_output=False)
one_hot_encoded = enc.fit_transform(final_race_results_df[['circuitName']])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=enc.get_feature_names_out(['circuitName']))

# Revisar índices y concater las columnas binarias, codificadas, al dataframe
one_hot_df.index = final_race_results_df.index
encoded_renamed_race_results_df = pd.concat([final_race_results_df, one_hot_df], axis=1)

# Dividir los datos en entrenamiento y prueba. Separamos por temporada
train_2014_2021 = encoded_renamed_race_results_df[(encoded_renamed_race_results_df['season'] >= 2014) & (encoded_renamed_race_results_df['season'] <= 2021)]
test_2022_2023 = encoded_renamed_race_results_df[(encoded_renamed_race_results_df['season'] >= 2022) & (encoded_renamed_race_results_df['season'] <= 2023)]

# Borrar columnas que no se van a usar para entrenar y probar el modelo
train = train_2014_2021.drop(columns=['season', 'circuitName'])
test = test_2022_2023.drop(columns=['season', 'circuitName'])

# FastestLapAvgSpeed es la variable a predecir, por ello, se borra del dataset del conjunto de entrenamiento
X_train = train.drop(columns=['FastestLapAvgSpeed']).values
y_train = train['FastestLapAvgSpeed'].values
X_test = test.drop(columns=['FastestLapAvgSpeed']).values
y_test = test['FastestLapAvgSpeed'].values

# Entrenar modelo
regr = LinearRegression()
regr.fit(X_train, y_train)

# Hacer predicciones sobre el conjunto de test
y_pred = regr.predict(X_test)

# Imprimir el Intercepto y los coeficientes
print(f'Intercept: {regr.intercept_.round(2)}')
print(f'Coef: {regr.coef_.round(2)}')
# Calcular RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse}')


##### Gráfico de dispersión para comparar datos reales (y_test) vs. datos predichos (y_pred)

In [ ]:

# Gráfico de dispersión para visualizar los datos de prueba vs las predicciones
plt.scatter(X_test[:, 2], y_test, c=X_test[:, 0], cmap='viridis', label='Test Data')
plt.colorbar(label="Grid Position")
plt.scatter(X_test[:, 2], y_pred, c='red', alpha=0.6, label='Predictions')
plt.xlabel("FastestLapTimeSeconds")
plt.ylabel("FastestLapAvgSpeed")
plt.title("Predictions vs Test Data (Color: Grid Position)")
plt.legend()
plt.show()


##### Gráfico con valores reales en eje 'x' y predichos en eje 'y'

In [ ]:
ax = sns.scatterplot(x=y_pred, y=y_test)
out = ax.axline((0, 0), slope=1, linewidth=.5, color='r', linestyle='dashed')

A partir de los datos mostrados y los gráficos, se puede concluir que las predicciones son muy cercanas a los valores reales. Además, se ha obtenido un RMSE entre 5-6, que se trata de un valor aceptable para el rango del valor independiente, que oscila mayoritariamente entre 200-300. Analizando los datos, se ha concluido que ese RMSE viene condicionado por circuitos cuyas condiciones climáticas han variado durante las diferentes temporadas, dando lugar a vueltas rápidas con tiempos superiores para malas condiciones climáticas. Objetar que este dato sobre el clima no se dispone en el conjunto de datos. También condiciona en los resultados tener menos cantidad de datos para ciertos circuitos porque el calendario cambia por cada temporada.

#### Linear Regression - Fastest Lap vs Fastest Lap Avg Speed (Modelo para un circuito) 

In [ ]:

# One-hot encoder para convertir variables categóricas en formato numérico para poder ser utilizadas por el modelo
enc = OneHotEncoder(sparse_output=False)
one_hot_encoded = enc.fit_transform(final_race_results_df[['circuitName']])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=enc.get_feature_names_out(['circuitName']))

# Revisar índices y concater las columnas binarias, codificadas, al dataframe
one_hot_df.index = final_race_results_df.index
encoded_renamed_race_results_df = pd.concat([final_race_results_df, one_hot_df], axis=1)

# Dividir los datos en entrenamiento y prueba. Separamos por temporada
train_2014_2021 = encoded_renamed_race_results_df[(encoded_renamed_race_results_df['season'] >= 2014) & (encoded_renamed_race_results_df['season'] <= 2021)]
test_2022_2023 = encoded_renamed_race_results_df[(encoded_renamed_race_results_df['season'] >= 2022) & (encoded_renamed_race_results_df['season'] <= 2023)]

# Seleccionar datos para un circuito concreto
train_2014_2021 = train_2014_2021[(train_2014_2021['circuitName'] == 'Circuit Paul Ricard')]
test_2022_2023 = test_2022_2023[(test_2022_2023['circuitName'] == 'Circuit Paul Ricard')]

# Borrar columnas que no se van a usar para entrenar y probar el modelo
train = train_2014_2021.drop(columns=['season', 'circuitName'])
test = test_2022_2023.drop(columns=['season', 'circuitName'])

# FastestLapAvgSpeed es la variable a predecir, por ello, se borra del dataset del conjunto de entrenamiento
X_train = train.drop(columns=['FastestLapAvgSpeed']).values
y_train = train['FastestLapAvgSpeed'].values
X_test = test.drop(columns=['FastestLapAvgSpeed']).values
y_test = test['FastestLapAvgSpeed'].values

# Entrenar modelo
regr = LinearRegression()
regr.fit(X_train, y_train)

# Hacer predicciones sobre el conjunto de test
y_pred = regr.predict(X_test)

# Imprimir el Intercepto y los coeficientes
print(f'Intercept: {regr.intercept_.round(2)}')
print(f'Coef: {regr.coef_.round(2)}')

# Calcular RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse}')


##### Gráfico de dispersión para comparar datos reales (y_test) vs. datos predichos (y_pred)

In [ ]:
plt.scatter(X_test[:, 2], y_test, c=X_test[:, 0], cmap='viridis', label='Test Data')
plt.colorbar(label="Grid Position")
plt.scatter(X_test[:, 2], y_pred, c='red', alpha=0.6, label='Predictions')
plt.xlabel("FastestLapTimeSeconds")
plt.ylabel("FastestLapAvgSpeed")
plt.title("Predictions vs Test Data (Color: Grid Position)")
plt.legend()
plt.show()

##### Gráfico con valores reales en eje 'x' y predichos en eje 'y'

In [ ]:
ax = sns.scatterplot(x=y_pred, y=y_test)
out = ax.axline((0, 0), slope=1, linewidth=.5, color='r', linestyle='dashed')

A partir de los datos mostrados y los gráficos, se puede concluir que las predicciones prácticamente coinciden a los valores reales. Se ha obtenido un RMSE muy próximo al cero, lo que indica que las predicciones del modelo están muy cerca de los valores reales, lo que representa un buen desempeño del modelo.

#### Random forest - Fastest Lap vs Fastest Lap Avg Speed (Todos los circuitos) 

In [ ]:
# One-hot encoder para convertir variables categóricas en formato numérico para poder ser utilizadas por el modelo
enc = OneHotEncoder(sparse_output=False)
one_hot_encoded = enc.fit_transform(final_race_results_df[['circuitName']])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=enc.get_feature_names_out(['circuitName']))

# Revisar índices y concater las columnas binarias, codificadas, al dataframe
one_hot_df.index = final_race_results_df.index
encoded_renamed_race_results_df = pd.concat([final_race_results_df, one_hot_df], axis=1)

# Dividir los datos en entrenamiento y prueba. Separamos por temporada
train_2014_2021 = encoded_renamed_race_results_df[(encoded_renamed_race_results_df['season'] >= 2014) & (encoded_renamed_race_results_df['season'] <= 2021)]
test_2022_2023 = encoded_renamed_race_results_df[(encoded_renamed_race_results_df['season'] >= 2022) & (encoded_renamed_race_results_df['season'] <= 2023)]

# Borrar columnas que no se van a usar para entrenar y probar el modelo
train = train_2014_2021.drop(columns=['circuitName'])
test = test_2022_2023.drop(columns=['circuitName'])

# FastestLapAvgSpeed es la variable a predecir, por ello, se borra del dataset del conjunto de entrenamiento
X_train = train.drop(columns=['FastestLapAvgSpeed']).values
y_train = train['FastestLapAvgSpeed'].values
X_test = test.drop(columns=['FastestLapAvgSpeed']).values
y_test = test['FastestLapAvgSpeed'].values


# Entrenar modelo
# rf = RandomForestRegressor(max_depth=10) # RMSE 9
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

# Obtener predicciones
y_train_pred = rf.predict(X_train)
y_pred = rf.predict(X_test)

# Calcular el RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE:', round(rmse, 2))

##### Gráfico con valores reales en eje 'x' y predichos en eje 'y'

In [ ]:
ax = sns.scatterplot(x=y_pred, y=y_test)
out = ax.axline((0, 0), slope=1, linewidth=.5, color='r', linestyle='dashed')

Las conclusiones a las que se puede llegar tras probar un Random Forest entrenado para todos los circuitos son prácticamente las mismas que para el modelo de regresión lineal, con un RMSE muy similar. No obstante, se ha probado a ajustar los distintos parámetros del RandomForestRegressor, como el max_depth o min_samples_leaf, y no se ha conseguido mejorar el modelo sino en ocasiones todo lo contrario, como se puede ver comentado en el código del modelo cuando he establecido un max_depth=10, obteniendo un RMSE = 9.

#### Random Forest - Fastest Lap vs Fastest Lap Avg Speed (Modelo para un circuito) 

In [ ]:
# One-hot encoder para convertir variables categóricas en formato numérico para poder ser utilizadas por el modelo
enc = OneHotEncoder(sparse_output=False)
one_hot_encoded = enc.fit_transform(final_race_results_df[['circuitName']])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=enc.get_feature_names_out(['circuitName']))

# Revisar índices y concater las columnas binarias, codificadas, al dataframe
one_hot_df.index = final_race_results_df.index
encoded_renamed_race_results_df = pd.concat([final_race_results_df, one_hot_df], axis=1)

# Dividir los datos en entrenamiento y prueba. Separamos por temporada
train_2014_2021 = encoded_renamed_race_results_df[(encoded_renamed_race_results_df['season'] >= 2014) & (encoded_renamed_race_results_df['season'] <= 2021)]
test_2022_2023 = encoded_renamed_race_results_df[(encoded_renamed_race_results_df['season'] >= 2022) & (encoded_renamed_race_results_df['season'] <= 2023)]

# Seleccionar datos para un circuito concreto
train_2014_2021 = train_2014_2021[(train_2014_2021['circuitName'] == 'Bahrain International Circuit')]
test_2022_2023 = test_2022_2023[(test_2022_2023['circuitName'] == 'Bahrain International Circuit')]

# Borrar columnas que no se van a usar para entrenar y probar el modelo
train = train_2014_2021.drop(columns=['circuitName'])
test = test_2022_2023.drop(columns=['circuitName'])

# FastestLapAvgSpeed es la variable a predecir, por ello, se borra del dataset del conjunto de entrenamiento
X_train = train.drop(columns=['FastestLapAvgSpeed']).values
y_train = train['FastestLapAvgSpeed'].values
X_test = test.drop(columns=['FastestLapAvgSpeed']).values
y_test = test['FastestLapAvgSpeed'].values

# Entrenar modelo
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

# Obtener predicciones
y_train_pred = rf.predict(X_train)
y_pred = rf.predict(X_test)

# Calcular el RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE:', round(rmse, 2))

##### Gráfico con valores reales en eje 'x' y predichos en eje 'y'

In [ ]:
ax = sns.scatterplot(x=y_pred, y=y_test)
out = ax.axline((0, 0), slope=1, linewidth=.5, color='r', linestyle='dashed')

El RMSE de entorno 0.1 obtenido al limitar los datos a un circuito específico muestra que el modelo tiene un gran poder predictivo en este contexto particular, lo que puede sugerir además, que el modelo captura correctamente las relaciones subyacentes dentro del conjunto de datos con el que se ha entrado y probado.


#### Nota final

Como mejora, se consideraría uncluir las circunstancia meteorológicas en el conjunto de datos y probar con circuitos que han estado presentes en el calendario desde 2014 hasta 2023 ininterrumpidamente.